In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'],access_type='offline',include_granted_scopes='true')
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=jzRFfldEfcEtXyinc0vCByd6qeYuZQ&access_type=offline&include_granted_scopes=true


In [4]:
import webbrowser
webbrowser.open(auth_url)

True

In [5]:
code = '4/Ti769qcLFSUBbCDU9dugj5EorHwFhcS9hPXGNIkVv70'

In [6]:
token = session.fetch_token(keychain['youtube']['token_uri'],
                            code=code,
                            client_secret=keychain['youtube']['client_secret'])  

In [7]:
print(token)

{'access_token': 'ya29.GlsbBS4zRWAXYJDYdpkxaZ77Xlr7mzw9z0uP59HWRw4B3jtzTRywqD9TtP81jsXFHmJ8kQlbHVvvHcF_DjiQo-piPnCAXPC_ePmhov48gg4jJ8DbjY-yg9PrC_eH', 'expires_in': 3600, 'refresh_token': '1/TJu4hZZeRWCYP9s3lERdkoorXY-nrWb1rFmEbCTvuMECXnp4xdpH4MRBPygubbs_', 'token_type': 'Bearer', 'expires_at': 1512692265.1297445}


In [8]:
import json
with open('token.json', 'w') as outfile:
    json.dump(token, outfile)

In [9]:
token=open('token.json',mode='r')
token=token.read()
token

'{"access_token": "ya29.GlsbBS4zRWAXYJDYdpkxaZ77Xlr7mzw9z0uP59HWRw4B3jtzTRywqD9TtP81jsXFHmJ8kQlbHVvvHcF_DjiQo-piPnCAXPC_ePmhov48gg4jJ8DbjY-yg9PrC_eH", "expires_in": 3600, "refresh_token": "1/TJu4hZZeRWCYP9s3lERdkoorXY-nrWb1rFmEbCTvuMECXnp4xdpH4MRBPygubbs_", "token_type": "Bearer", "expires_at": 1512692265.1297445}'

In [10]:
token=json.loads(token)
token

{'access_token': 'ya29.GlsbBS4zRWAXYJDYdpkxaZ77Xlr7mzw9z0uP59HWRw4B3jtzTRywqD9TtP81jsXFHmJ8kQlbHVvvHcF_DjiQo-piPnCAXPC_ePmhov48gg4jJ8DbjY-yg9PrC_eH',
 'expires_at': 1512692265.1297445,
 'expires_in': 3600,
 'refresh_token': '1/TJu4hZZeRWCYP9s3lERdkoorXY-nrWb1rFmEbCTvuMECXnp4xdpH4MRBPygubbs_',
 'token_type': 'Bearer'}

In [11]:
importlib.reload(keys)
keychain = keys.keychain
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [12]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import io
D = {}
D['access_token'] = token['access_token']

In [13]:
data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
def getData(data,vid):
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(item['snippet']['publishedAt'])

In [14]:
url ='https://www.googleapis.com/youtube/v3/videos'
D['part'] = ['snippet,contentDetails,statistics']
D['maxResults'] = '50'
D['chart'] ='mostPopular'
popular_vid = requests.get(url, params = D)
popular_vid = popular_vid.json()

In [15]:
page = []
data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
for i in range(20):
    if 'nextPageToken' not in popular_vid:
        nex_page = 'CAUQAA'
    else:
        next_page = popular_vid['nextPageToken']
    page.append(next_page)
    D['pageToken'] = next_page
    popular_vid = requests.get(url, params = D)
    popular_vid = popular_vid.json()
    getData(data,popular_vid['items'])
df= pd.DataFrame(data)
df[:10]

,categoryId,channelId,channelTitle,publishedAt,videoId
0,22,UCGaVdbSav8xWuFWTadK6loA,vlogbrothers,2017-12-05T23:16:00.000Z,Rjv1NlMiUyI
1,25,UCqnbDFdCpuN8CMEg0VuEBqA,The New York Times,2017-12-06T02:30:01.000Z,Hi91aDmYI2E
2,24,UCSV8iMrDMdzc79kPCS9qucg,The Talk,2017-12-05T22:00:00.000Z,032BPsxhreM
3,25,UCkH1uDkyuO9sVjSqdqBygOg,CBS Los Angeles,2017-12-06T01:29:41.000Z,UZf9m_x7o88
4,26,UCbpMy0Fg74eXXkvxJrtEn3w,Bon Appétit,2017-12-06T17:01:10.000Z,vgX5M1eVhyw
5,25,UCIRYBXDze5krPDzAEOxFGVA,Guardian News,2017-12-05T19:35:27.000Z,sciODVQP43w
6,24,UCZ1Sc5xjWpUnp_o_lUTkvgQ,The Grand Tour,2017-12-06T12:58:36.000Z,aJ9XruZm_Hg
7,22,UCIiBf-JbtCazHSFqXV4JgoA,Grace Helbig,2017-12-05T20:00:04.000Z,v_CMMWCN5nQ
8,25,UCY0YIply-je0EhSWLgpftVw,Beme News,2017-12-06T22:00:01.000Z,NbUtY1RVvn8
9,24,UCPWQWav6BpPvtanCtloXkiw,The CW Television Network,2017-12-07T06:00:01.000Z,xkYpO9ATUT0


In [16]:
url = 'https://www.googleapis.com/youtube/v3/videoCategories'
C = {}
C['access_token'] = token['access_token']
C['part'] = 'snippet'
C['regionCode'] = 'US'
category_vid = requests.get(url, params = C)
category_vid = category_vid.json()

In [17]:
data_category = {'id':[],'title':[]}
def categoryTable(data,json):
    for item in json:
        data['id'].append(item['id'])
        data['title'].append(item['snippet']['title'])
categoryTable(data_category,category_vid['items'])
df1 = pd.DataFrame(data_category)
df1[:10]

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [18]:
result = pd.merge(df, df1, left_on='categoryId', right_on='id')
result.to_csv('result.csv')

In [19]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

In [20]:
print(token)

{'access_token': 'ya29.GlsbBXO6vHqp7Virt1WSE7lx1tzOZA8fpsbjdth5uMxKbRB-aYO64dHLnNxwn-sqaE9T394h6onER-8bhwiT6YxaQarJ4JhAvFguImSTumhg6qO5KZmLISHDeNwk', 'expires_in': 3600, 'token_type': 'Bearer', 'expires_at': 1512692336.888527, 'refresh_token': '1/TJu4hZZeRWCYP9s3lERdkoorXY-nrWb1rFmEbCTvuMECXnp4xdpH4MRBPygubbs_'}
